<a href="https://colab.research.google.com/github/KamilBienias/data-science/blob/main/kursPawe%C5%82Krakowiak/machine-learning-bootcamp/supervised/05_case_studies/03_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### scikit-learn
Strona biblioteki: [https://scikit-learn.org](https://scikit-learn.org)  

Dokumentacja/User Guide: [https://scikit-learn.org/stable/user_guide.html](https://scikit-learn.org/stable/user_guide.html)

Podstawowa biblioteka do uczenia maszynowego w języku Python.

Aby zainstalować bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install scikit-learn
```
Aby zaktualizować do najnowszej wersji bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install --upgrade scikit-learn
```
Kurs stworzony w oparciu o wersję `0.22.1`

### Spis treści:
1. [Import bibliotek](#0)
2. [Wygenerowanie danych](#1)
3. [Wektoryzacja tekstu](#2)
4. [Wektoryzacja tekstu - bigramy](#3)
5. [TFIDF Transformer](#4)
6. [TFIDF Vectorizer](#5)
7. [Przygotowanie danych tekstowych - przykład](#6)



### <a name='0'></a> Import bibliotek

In [1]:
print("############################################################")
print("Etap23. Odcinek: Praca z danymi tekstowymi - przygotowanie do modelu")

import numpy as np
import pandas as pd
import plotly.express as px
import sklearn

np.random.seed(42)
np.set_printoptions(precision=6, suppress=True, edgeitems=10, linewidth=1000, formatter=dict(float=lambda x: f'{x:.2f}'))
sklearn.__version__

############################################################
Etap23. Odcinek: Praca z danymi tekstowymi - przygotowanie do modelu


'0.22.2.post1'

### <a name='1'></a> Wygenerowanie danych

In [2]:
documents = [
    'Today is Friday',
    'I like Friday',
    'Today I am going to learn Python.',
    'Friday, Friday!!!'
]

print(documents)

['Today is Friday', 'I like Friday', 'Today I am going to learn Python.', 'Friday, Friday!!!']


### <a name='2'></a> Wektoryzacja tekstu

In [3]:
# klasa CountVectorizer bierze tylko słowa (usuwa znaki interpunkcyjne)
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(documents)

# w rezultacie dostajemy macierz rzadką sparse o wymiarach 4 (ilość zdań) na 9 (ilość unikalnych słów).
# Zwraca macierz rzadką bo te macierze zawierają dużo zer, więc oszczędza pamięć 
# (nie chcemy numpy ndarray, która ma zera)

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [4]:
# podglądam tablicę częstości zamieniając ją najpierw na numpy array (metodą toarray)
vectorizer.fit_transform(documents).toarray()
# cztery listy bo 4 zdania. Liczby to liczebność słowa w danym zdaniu. Lista słów pod spodem.

array([[0, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 1, 1],
       [0, 2, 0, 0, 0, 0, 0, 0, 0]])

In [5]:
# słowa z tekstu są wystandaryzowane i ułożone w sposób alfabetyczny
vectorizer.get_feature_names()
# pierwsze zdanie ma słowa friday, is, today więc tam są jedynki.
# Ostatnie zdanie ma dwa raz friday więc jest 2

['am', 'friday', 'going', 'is', 'learn', 'like', 'python', 'to', 'today']

In [6]:
df = pd.DataFrame(data=vectorizer.fit_transform(documents).toarray(), 
                  columns=vectorizer.get_feature_names())

df

,am,friday,going,is,learn,like,python,to,today
0,0,1,0,1,0,0,0,0,1
1,0,1,0,0,0,1,0,0,0
2,1,0,1,0,1,0,1,1,1
3,0,2,0,0,0,0,0,0,0


In [10]:
# atrybut vocabulary_ przetrzymuje słownik słów
# Mapuje wyraz na indeks z tablicy numpy ndarray
vectorizer.vocabulary_

{'am': 0,
 'friday': 1,
 'going': 2,
 'is': 3,
 'learn': 4,
 'like': 5,
 'python': 6,
 'to': 7,
 'today': 8}

In [7]:
# samą metodą transform dopasowuję do insniejących wyrazów kodowanie zdania 'Friday morning'
# Zauważmy że nie uczyła się słowa morning, więc go nie ma (jest tylko 1 tam gdzie był friday).
vectorizer.transform(['Friday morning']).toarray()

array([[0, 1, 0, 0, 0, 0, 0, 0, 0]])

### <a name='3'></a> Wektoryzacja tekstu - bigramy

In [8]:
# bigram to dwa wyrazy połączone w bliskim sąsiedztwie
# parametr ngram_range to tupla (1,2)
bigram = CountVectorizer(ngram_range=(1, 2), min_df=1)    # min_df=2
# dopasowuje i przetransformuje dokument do tablicy ndarray
bigram.fit_transform(documents).toarray()

array([[0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [11]:
# Mapuje wyraz bigram na indeks z tablicy numpy ndarray
bigram.vocabulary_

{'am': 0,
 'am going': 1,
 'friday': 2,
 'friday friday': 3,
 'going': 4,
 'going to': 5,
 'is': 6,
 'is friday': 7,
 'learn': 8,
 'learn python': 9,
 'like': 10,
 'like friday': 11,
 'python': 12,
 'to': 13,
 'to learn': 14,
 'today': 15,
 'today am': 16,
 'today is': 17}

In [12]:
df = pd.DataFrame(data=bigram.fit_transform(documents).toarray(), 
                  columns=bigram.get_feature_names())
df

,am,am going,friday,friday friday,going,going to,is,is friday,learn,learn python,like,like friday,python,to,to learn,today,today am,today is
0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1
1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
2,1,1,0,0,1,1,0,0,1,1,0,0,1,1,1,1,1,0
3,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### <a name='4'></a> TFIDF Transformer (metoda obliczania ważności słów w dokumencie w oparciu o częstość wystąpień)

In [13]:
# jeśli słowo występuje w każdym zdaniu to ma mniejszą ważność. Unikalne słowa mają większą wartość
documents = [
    'Friday morning',
    'Friday chill',
    'Friday - morning',
    'Friday, Friday morning!!!'
]

print(documents)

['Friday morning', 'Friday chill', 'Friday - morning', 'Friday, Friday morning!!!']


In [14]:
counts = vectorizer.fit_transform(documents).toarray()
counts

array([[0, 1, 1],
       [1, 1, 0],
       [0, 1, 1],
       [0, 2, 1]])

In [15]:
df = pd.DataFrame(data=vectorizer.fit_transform(documents).toarray(), columns=vectorizer.get_feature_names())
df

,chill,friday,morning
0,0,1,1
1,1,1,0
2,0,1,1
3,0,2,1


In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
tfidf.fit_transform(counts).toarray()

# słowo friday w pierwszym dokumencie ma mniejszą wartość niż morning w pierwszym dokumencie
# bo friday we wszystkich dokumentach było częściej niż morning we wszystkich dokumentach

array([[0.00, 0.63, 0.77],
       [0.89, 0.46, 0.00],
       [0.00, 0.63, 0.77],
       [0.00, 0.85, 0.52]])

### <a name='5'></a> TFIDF Vectorizer (gotowa klasa, żeby nie robić najpierw CountVectorizer)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit_transform(documents).toarray()
# od razu ten wynik

array([[0.00, 0.63, 0.77],
       [0.89, 0.46, 0.00],
       [0.00, 0.63, 0.77],
       [0.00, 0.85, 0.52]])

In [19]:
tfidf_vectorizer.idf_

array([1.92, 1.00, 1.22])

### <a name='6'></a> Przygotowanie danych tekstowych - przykład

In [21]:
# pobiera 20 tematycznych grup
from sklearn.datasets import fetch_20newsgroups

In [22]:
# dane tekstowe dotyczące grafiki komputerowej. Będą to maile
raw_data = fetch_20newsgroups(subset='train', categories=['comp.graphics'], random_state=42)
raw_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [23]:
all_data = raw_data.copy()
# 5 pierwszych próbek (które są mailami)
all_data['data'][:5]

["From: bbs.mirage@tsoft.net (Jerry Lee)\nSubject: Cobra 2.0 1-b-1 Video card HELP ME!!!!\nOrganization: The TSoft BBS and Public Access Unix, +1 415 969 8238\nLines: 22\n\nDoes ANYONE out there in Net-land have any information on the Cobra 2.20 \ncard?  The sticker on the end of the card reads\n        Model: Cobra 1-B-1\n        Bios:  Cobra v2.20\n\nI Havn't been able to find anything about it from anyone!  If you have \nany information on how to get a hold of the company which produces the \ncard or know where any drivers are for it, PLEASE let me know!\n\nAs far as I can tell, it's a CGA card that is taking up 2 of my 16-bit \nISA slots but when I enable the test patterns, it displays much more than \nthe usualy 4 CGA colors... At least 16 from what I can count.. Thanks!\n\n              .------------------------------------------.\n              : Internet: jele@eis.calstate.edu          :\n              :           bbs.mirage@gilligan.tsoft.net  :\n              :           bbs.

In [24]:
# pierwszy przykładowy mail
print(all_data['data'][0])

From: bbs.mirage@tsoft.net (Jerry Lee)
Subject: Cobra 2.0 1-b-1 Video card HELP ME!!!!
Organization: The TSoft BBS and Public Access Unix, +1 415 969 8238
Lines: 22

Does ANYONE out there in Net-land have any information on the Cobra 2.20 
card?  The sticker on the end of the card reads
        Model: Cobra 1-B-1
        Bios:  Cobra v2.20

I Havn't been able to find anything about it from anyone!  If you have 
any information on how to get a hold of the company which produces the 
card or know where any drivers are for it, PLEASE let me know!

As far as I can tell, it's a CGA card that is taking up 2 of my 16-bit 
ISA slots but when I enable the test patterns, it displays much more than 
the usualy 4 CGA colors... At least 16 from what I can count.. Thanks!

              .------------------------------------------.
              : Internet: jele@eis.calstate.edu          :
              :           bbs.mirage@gilligan.tsoft.net  :
              :           bbs.mirage@tsoft.sf-bay.org

In [25]:
# jedna nazwa klasy
all_data['target_names']

['comp.graphics']

In [26]:
all_data['target'][:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [27]:
tfidf = TfidfVectorizer()
# metoda fit_transform dopasowuje macierz rzadką, dopiero potem wyświetlam jako numpy array
tfidf.fit_transform(all_data['data']).toarray()

array([[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, ..., 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, ..., 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.04, 0.00, 0.00],
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, ..., 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, ..., 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, ..., 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, ..., 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, ..., 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 